In [ ]:
import datetime
import re
import sys
from datetime import date

from langchain_community.chat_models import ChatTongyi
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.tools import StructuredTool, tool
from pydantic import BaseModel, Field

sys.path.append('e:/code/python/WangXiang')
chatllm = ChatTongyi(model='qwen-max')

In [11]:
from citra.mcp.db_utils import get_connection

conn = get_connection()

数据库连接成功


In [ ]:
@tool
def collect_outage_info(
    line_name: str, occur_time: date = date(1000, 1, 1), end_time: date = date(3000, 12, 31)
) -> str:
    """停电明细（汇总）查询。获取某条线路(line_name)在过去一段时间（start_time，end_time）内属于某个供电所（unit_name）的停电汇总信息。"""
    # cursor.execute(f'select count(*)  from t_event_alarm_inter where lineName={line_name} LIMIT 1')
    # if cursor.fetchone() is None:
    #     return f'线路{line_name}不存在，请检查线路名称'
    with conn.cursor() as cursor:
        query = f"select count(*)  from t_event_alarm_inter where  occurTime>='{occur_time}' and (endTime<='{end_time}' or endTime is null)"
        cursor.execute(query)
        count = cursor.fetchall()[0][0]
    return f'{line_name}在{occur_time}到{end_time}间的停电次数为：{count}'


@tool
def count_outage(line_name: str, occur_time: date = date(1000, 1, 1), end_time: date = date(3000, 12, 31)) -> str:
    """停电次数查询。计数某条线路(line_name)在过去一段时间（start_time，end_time）的停电次数。"""
    today = datetime.date.today().strftime('%Y-%m-%d')
    query = f"select  COUNT(*) as count from t_event_alarm_inter where lineName LIKE '{line_name}%' and occurTime>='{today}' "

    with conn.cursor() as cursor:
        cursor.execute(query)
        count = cursor.fetchall()[0][0]
    return f'{line_name}今天的停电次数为：{count}'


In [ ]:
def create_sql(question: str) -> str:
    query = (
        str(
            chatllm.invoke(f"""编写一条 SQL 查询语句，仅输出 SQL，不要输出解释说明。表结构：CREATE TABLE \
                            t_event_alarm_inter (equipName设备名称,lineName线路名称,faultType停电类型,\
                           occurTime停电开始时间,endTime停电结束时间,
                            gdsName 供电所单位名称，unitName供电公司名称。问题：{question} """)
        )
        .replace('`', '')
        .replace('\\n', ' ')
    )
    search = re.search(r'((select|SELECT)[^"]+)', query)
    if search:
        return search.group()
    return ''


In [ ]:
def execute_sql(sql: str):
    with conn.cursor() as cursor:
        cursor.execute(sql)
        results = cursor.fetchall()
        result_strings = []
        columns = [desc[0] for desc in cursor.description]
        result_strings.append(' '.join(columns))
        for row in results:
            row_str = ' '.join(str(item) for item in row)
            result_strings.append(row_str)

        # 最后拼接成一个字符串，用换行符隔开
        final_result = '\n'.join(result_strings)

        return final_result

In [210]:
sq = create_sql('三门供电公司按供电所维度统计本月配变停电次数')
print(sq)
print(execute_sql(sq))

SELECT gdsName, COUNT(*) AS powerOutageCount FROM t_event_alarm_inter WHERE unitName = '三门供电公司'  AND occurTime >= DATE_FORMAT(CURDATE(), '%Y-%m-01') AND occurTime < DATE_FORMAT(CURDATE() + INTERVAL 1 MONTH, '%Y-%m-01') GROUP BY gdsName; 
gdsName powerOutageCount
健跳供电所 29
浦坝港供电所 11
滨海供电所 17
珠岙供电所 59
花桥供电所 109


渔港S400线 专变停电 2025-04-29 10:42:00 None 新港供电所
兴港S501线 专变停电 2025-04-29 08:57:47 None 新港供电所
纪海S215线 专变停电 2025-04-30 12:30:55 2025-04-30 12:51:30 新港供电所
蔡家S676线 公变停电 2025-04-30 02:42:42 2025-04-30 03:00:15 港城供电所
蔡家S676线 公变停电 2025-04-30 02:42:18 2025-04-30 03:00:39 港城供电所
蔡家S676线 公变停电 2025-04-30 02:41:06 2025-04-30 03:06:36 港城供电所
蔡家S676线 公变停电 2025-04-30 02:41:05 2025-04-30 03:00:26 港城供电所
蔡家S676线 专变停电 2025-04-30 02:41:04 2025-04-30 03:05:45 港城供电所
蔡家S676线 专变停电 2025-04-30 02:41:02 2025-04-30 03:05:43 港城供电所
蔡家S676线 专变停电 2025-04-30 02:41:01 2025-04-30 03:05:43 港城供电所
蔡家S676线 专变停电 2025-04-30 02:41:00 2025-04-30 03:05:00 港城供电所
蔡家S676线 专变停电 2025-04-30 02:41:00 2025-04-30 03:05:00 港城供电所
蔡家S676线 专变停电 2025-04-30 02:41:00 2025-04-30 03:06:48 港城供电所
蔡家S676线 专变停电 2025-04-30 02:40:59 2025-04-30 03:23:58 港城供电所
蔡家S676线 公变停电 2025-04-30 02:40:55 2025-04-30 03:06:34 港城供电所
蔡家S676线 公变停电 2025-04-30 02:40:00 2025-04-30 02:59:00 港城供电所
蔡家S676线 公变停电 2025-04-30 02:40:00 2025-04-30 03:00:05 港城供电所
蔡家S676线 公变停电 2025-04-30 02:

In [ ]:
请问中市S487线3个月内停电次数

In [ ]:
ask_sql('请问中市S487线3个月内停电次数')

OperationalError: (1305, 'FUNCTION wj2.DATEADD does not exist')

In [ ]:
query = (
    str(
        chatllm.invoke("""编写一条 SQL 查询语句，仅输出 SQL，不要输出任何解释说明。表结构定义为CREATE TABLE t_event_alarm_inter (
                        lineName 线路名称, faultType 停电类型,   occurTime 停电时间,  endTime 恢复时间,
      gdsName 供电所单位名称，  unitName供电公司名称。请问龙港一年内停电明细情况 """)
    )
    .replace('`', '')
    .replace('\\n', ' ')
)

In [ ]:
@tool
def extract_args(line_name: str, fault_type: str, unit_name: str, occur_time: date, end_time: date) -> str:
    """线路和时间的停电查询。获取用户提问，填充sql查询语句的select字段和where条件字段。"""
    base_query = 'select * from t_event_alarm_inter where 1=1'
    if line_name:
        base_query += f" and lineName LIKE '{line_name}%'"
    if fault_type:
        base_query += f" and faultType LIKE '{fault_type}%'"
    if unit_name:
        base_query += f" and unitName LIKE '{unit_name}%'"
    if occur_time:
        base_query += f" and occurTime>='{occur_time}'"
    if end_time:
        base_query += f" and (endTime<='{end_time}' or endTime is null)"
    return base_query


class SqlArgs(BaseModel):
    line_name: str = Field(description='停电线路名称')
    fault_type: str = Field(description='停电类型')
    unit_name: str = Field(description='供电公司名称')
    occur_time: date = Field(date(1000, 1, 1), description='停电开始时间')
    end_time: date = Field(date(3000, 1, 1), description='停电结束时间')


sql_args = StructuredTool.from_function(
    func=extract_args,
    name='extractor',
    description='根据用户提问，提取信息至sql查询参数',
    args_schema=SqlArgs,
    return_direct=True,
)

# type: ignore
tools = [count_outage, collect_outage_info]
chat_with_tools = chatllm.bind_tools(tools)


def call_tools(msg: AIMessage) -> list[dict]:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call['output'] = tool_map[tool_call['name']].invoke(tool_call['args'])
    return tool_calls


chain = chat_with_tools | call_tools


In [59]:
chat_with_tools.invoke('请问中市S487线1年内的停电天数?').tool_calls[0]

{'name': 'count_outage',
 'args': {'line_name': '中市S487线',
  'start_time': '2021-06-01',
  'end_time': '2022-06-01'},
 'id': 'call_26679a5b4e8e401cb1556c',
 'type': 'tool_call'}

In [60]:
def ask_outage(question: str) -> str:
    """Ask a question to the model and get the answer."""
    now = datetime.date.today().strftime('%Y-%m-%d')
    answer = chat_with_tools.invoke([HumanMessage(content=f'今天日期是{now},{question}')])
    result = call_tools(answer)
    return result[0]['output']

In [ ]:
messages = [HumanMessage(content='请问龙港今天停电情况')]
ai_mes = chat_with_tools.invoke(messages)
print(ai_mes)
messages.append(ai_mes)


content='' additional_kwargs={'tool_calls': [{'function': {'name': 'count_outage_today', 'arguments': '{"unit_name": "龙港"}'}, 'index': 0, 'id': 'call_83ba090c12c14f1f9d4cb5', 'type': 'function'}]} response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '8d1372c6-6beb-98fe-87b9-c48340dbc924', 'token_usage': {'input_tokens': 319, 'output_tokens': 21, 'total_tokens': 340, 'prompt_tokens_details': {'cached_tokens': 0}}} id='run--be0b5a77-cbb8-4c2b-ac7f-a48d5d1d8fe3-0' tool_calls=[{'name': 'count_outage_today', 'args': {'unit_name': '龙港'}, 'id': 'call_83ba090c12c14f1f9d4cb5', 'type': 'tool_call'}]


In [78]:
chain.invoke([HumanMessage(content='请问龙港今天停电情况')])

[{'name': 'count_outage_today',
  'args': {'unit_name': '龙港'},
  'id': 'call_d590d0b6f7e840e5875b38',
  'type': 'tool_call',
  'output': '龙港今天的停电次数为：3'}]

In [61]:
ask_outage('请问龙港当日停电情况')

'龙港在2025-05-09到2025-05-09间的停电次数为：5'